In [16]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-a0wjp0i3
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-a0wjp0i3
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=e5915a630024d74a702985496cc452f233bd7c8459363151b49b0a93ca7116b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ocw5xw6b/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [18]:
%%cu
#include <stdio.h>

#define M 3
#define N 4


__global__ void mult_cols_on_threads (int *A, int *B, int *C) {
    // Get the 1D Array index of the matrix
    int idx = threadIdx.x;
    printf("idx = %d\n", idx);
    for (int i = 0; i < N; ++i, idx += M) {
        C[idx] = A[idx] * B[idx];
    }
}

__global__ void mult_elem_on_threads (int *A, int *B, int *C) {
    // Get the 1D Array index of the matrix
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    printf("idx = %d\n", idx);
    C[idx] = A[idx] * B[idx];
}

__global__ void mult_rows_on_threads (int *A, int *B, int *C) {
    // Get the 1D Array index of the matrix
    int idx = threadIdx.x;
    printf("idx = %d\n", idx);
    for (int i = 0; i < M; ++i) {
        C[i + M * idx] = A[i + M * idx] * B[i + M * idx];
    }
}

int main () {
    
    int A[M * N], B[M * N], C[M * N],D[M*N],E[M*N];

    int i, j;
    for (i = 0; i < M * N; ++i) {
        A[i] = i + 1;
        B[i] = M * N - i - 1;
    }

    int *d_a, *d_b, *d_c ,*d_d ,*d_e;

    int size = sizeof(int) * M * N;

    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);
    cudaMalloc((void**)&d_d, size);
    cudaMalloc((void**)&d_e, size);

    cudaMemcpy(d_a, &A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &B, size, cudaMemcpyHostToDevice);

    mult_cols_on_threads<<<1, N>>>(d_a, d_b, d_c);
    mult_elem_on_threads<<<M,N>>>(d_a,d_b,d_d);
    mult_rows_on_threads<<<1,N>>>(d_a,d_b,d_e);

    // Copy the result back to the host
    cudaMemcpy(&C, d_c, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(&D, d_d, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(&E, d_e, size, cudaMemcpyDeviceToHost);


    printf("A:\n");
    for (i = 0; i < N; ++i) {
        for (j = 0; j < M; ++j) {
            printf("%d\t", A[i * M + j]);
        }
        printf("\n");
    }
    printf("\n");

    printf("B:\n");
    for (i = 0; i < N; ++i) {
        for (j = 0; j < M; ++j) {
            printf("%d\t", B[i * M + j]);
        }
        printf("\n");
    }
    printf("\n");

    printf("Part 1> A * B:\n");
    for (i = 0; i < N; ++i) {
        for (j = 0; j < M; ++j) {
            printf("%d\t", C[i * M + j]);
        }
        printf("\n");
    }
    printf("\n");
    
    printf("Part 2> A * B:\n");
    for (i = 0; i < N; ++i) {
        for (j = 0; j < M; ++j) {
            printf("%d\t", D[i * M + j]);
        }
        printf("\n");
    }
    printf("\n");
    
    printf("Part 3> A * B:\n");
    for (i = 0; i < N; ++i) {
        for (j = 0; j < M; ++j) {
            printf("%d\t", E[i * M + j]);
        }
        printf("\n");
    }
    printf("\n");

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_d);
    cudaFree(d_e);

    getchar();

    return 0;
}

idx = 0
idx = 1
idx = 2
idx = 3
idx = 0
idx = 1
idx = 2
idx = 3
idx = 8
idx = 9
idx = 10
idx = 11
idx = 4
idx = 5
idx = 6
idx = 7
idx = 0
idx = 1
idx = 2
idx = 3
A:
1	2	3	
4	5	6	
7	8	9	
10	11	12	

B:
11	10	9	
8	7	6	
5	4	3	
2	1	0	

Part 1> A * B:
11	20	27	
32	35	36	
35	32	27	
20	11	0	

Part 2> A * B:
11	20	27	
32	35	36	
35	32	27	
20	11	0	

Part 3> A * B:
11	20	27	
32	35	36	
35	32	27	
20	11	0	


